In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 한글폰트 설정
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

plt.rc('font', family='NanumGothic')
sns.set(font="NanumGothic",#"NanumGothicCoding",
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')

# 경고 표시 무시
import warnings
warnings.filterwarnings('ignore')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (4,395 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120874 files and dire

In [2]:
# 출력 옵션 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
# 구글 코랩 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# train 데이터 불러오기
train_data = pd.read_csv('/content/drive/MyDrive/미니프로젝트5차part2_Kaggle/train.csv')

In [ ]:
train_data.shape

(100000, 9)

In [ ]:
train_data['label'].value_counts()

2     12170
8     11497
1     11465
7     10594
10     9579
9      9282
11     9112
3      8305
4      7208
5      5411
6      5377
Name: label, dtype: int64

* 클래스 불균형이 존재하는 것으로 보인다. 훈련할 떄 stratify를 써야할 것 같은데<br> 그것이 클래스 균형을 이끌어낸다고 보장할 수 없다. 그냥 어느정도 클래스 균형 방향으로 갈 수 있도록 도와주는 것일 뿐...

In [ ]:
# test 데이터 불러오기
test_data = pd.read_csv('/content/drive/MyDrive/미니프로젝트5차part2_Kaggle/test.csv')

In [ ]:
test_data.shape

(13234, 8)

In [ ]:
# train 데이터에 대해 전반적인 탐색을 해야겠다.
train_data.head()

,Unnamed: 0,timestamp,A_x,A_y,A_z,B_x,B_y,B_z,label
0,0,2019-01-12 00:45:54.450,-0.259130,-0.834869,-0.485499,0.196409,NaN,0.384934,8
1,1,2000-01-01 01:37:06.440,0.370490,0.175042,0.122625,-0.338242,0.358245,0.126491,2
2,2,2019-01-12 00:45:33.900,-0.257837,-0.881947,-0.391895,0.196027,0.894537,0.411221,8
3,3,2000-01-01 00:46:22.680,-0.937753,-0.055961,0.362041,-0.929881,0.087673,0.134609,11
4,4,2000-01-01 00:49:56.620,-0.988320,-0.190390,0.157909,-0.954669,-0.024810,-0.388420,6


In [ ]:
# train 데이터에 'Unamed : 0' 삭제한다.
train_data.drop(['Unnamed: 0'], axis=1, inplace=True)

* train_data는 시계열 데이터인데, 시간순으로 정렬되어 있지 않아서 오름차순으로 정렬해야 한다.

* 시계열 데이터는 0.2, 0.4초 정도 각각 차이 나는 것 같다.
  * 나는 그점이 우려스러웠다. 뭐냐면 갑자기 시간 차이가 많이 날떄(ex.갑자기 2분, 3분, 30분 ...) 가 존재할 수가 있으면 어떻게 결측치를 처리할까 그런 고민을 한 적이 있었다.

In [ ]:
train_data.head()

,timestamp,A_x,A_y,A_z,B_x,B_y,B_z,label
0,2019-01-12 00:45:54.450,-0.259130,-0.834869,-0.485499,0.196409,NaN,0.384934,8
1,2000-01-01 01:37:06.440,0.370490,0.175042,0.122625,-0.338242,0.358245,0.126491,2
2,2019-01-12 00:45:33.900,-0.257837,-0.881947,-0.391895,0.196027,0.894537,0.411221,8
3,2000-01-01 00:46:22.680,-0.937753,-0.055961,0.362041,-0.929881,0.087673,0.134609,11
4,2000-01-01 00:49:56.620,-0.988320,-0.190390,0.157909,-0.954669,-0.024810,-0.388420,6


In [ ]:
train_data['timestamp'] = pd.to_datetime(train_data['timestamp']) # 'timestamp' 변수의 데이터타입을 datetime으로 변경

In [ ]:
train_data = train_data.sort_values(by='timestamp', ) # 'timestamp' 변수를 오름차순 정렬한다. 진정한 시계열 데이터에 맞게

In [ ]:
# index 번호를 다시 0부터 부여한다. (왜냐하면 'timestamp' 변수를 오름차순 정렬했기 떄문에 index 번호가 불규칙적으로 배치될 것이기 떄문이다.)
train_data = train_data.reset_index(drop=True)

* 시계열 데이터에 대한 결측치(NaN)을 어떻게 처리하느냐가 관건이다.
  * 'A_x' ~ 'B_z' 까지 결측치가 10000개 있어서 이를 어떻게 처리하느냐가 관건이다.

* 전략) 저는 결측치 기준으로 이전 값과 이후 값을 파악해서 평균 구한 값으로 대체하려고 합니다.

In [ ]:
train_data.info() # 변수에 대한 결측치를 확인한다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   timestamp  100000 non-null  datetime64[ns]
 1   A_x        90000 non-null   float64       
 2   A_y        90000 non-null   float64       
 3   A_z        90000 non-null   float64       
 4   B_x        90000 non-null   float64       
 5   B_y        90000 non-null   float64       
 6   B_z        90000 non-null   float64       
 7   label      100000 non-null  int64         
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 6.1 MB


In [ ]:
train_data.head(10)

,timestamp,A_x,A_y,A_z,B_x,B_y,B_z,label
0,2000-01-01 00:00:00.000,-0.988758,-0.144844,0.183325,-0.991851,-0.022659,-0.306466,6
1,2000-01-01 00:00:00.020,-1.008465,NaN,0.156743,-0.976224,-0.040058,-0.277258,6
2,2000-01-01 00:00:00.040,-0.991936,-0.143049,0.156618,-0.949758,-0.035796,-0.299548,6
3,2000-01-01 00:00:00.060,-1.007975,-0.140831,0.155302,-1.006806,-0.013057,-0.289020,6
4,2000-01-01 00:00:00.080,-0.991991,-0.159979,0.166965,NaN,0.015054,-0.293883,6
5,2000-01-01 00:00:00.100,-1.008148,-0.152922,0.171494,-0.967190,-0.041271,-0.289058,6
6,2000-01-01 00:00:00.120,-0.991536,-0.159341,0.160002,-0.936596,-0.070587,-0.294263,6
7,2000-01-01 00:00:00.180,-0.990727,-0.146829,NaN,-0.967847,-0.017197,-0.275129,6
8,2000-01-01 00:00:00.200,-0.976904,-0.131678,0.182595,-0.984224,-0.017629,-0.257675,6
9,2000-01-01 00:00:00.220,NaN,-0.145474,0.176887,-0.999342,-0.003938,-0.235726,6


In [ ]:
train_data.set_index('timestamp', inplace=True) # 'timestamp' 열을 인덱스로 설정

In [ ]:
train_data.head(10)

,A_x,A_y,A_z,B_x,B_y,B_z,label
timestamp,,,,,,,
2000-01-01 00:00:00.000,-0.988758,-0.144844,0.183325,-0.991851,-0.022659,-0.306466,6
2000-01-01 00:00:00.020,-1.008465,NaN,0.156743,-0.976224,-0.040058,-0.277258,6
2000-01-01 00:00:00.040,-0.991936,-0.143049,0.156618,-0.949758,-0.035796,-0.299548,6
2000-01-01 00:00:00.060,-1.007975,-0.140831,0.155302,-1.006806,-0.013057,-0.289020,6
2000-01-01 00:00:00.080,-0.991991,-0.159979,0.166965,NaN,0.015054,-0.293883,6
2000-01-01 00:00:00.100,-1.008148,-0.152922,0.171494,-0.967190,-0.041271,-0.289058,6
2000-01-01 00:00:00.120,-0.991536,-0.159341,0.160002,-0.936596,-0.070587,-0.294263,6
2000-01-01 00:00:00.180,-0.990727,-0.146829,NaN,-0.967847,-0.017197,-0.275129,6
2000-01-01 00:00:00.200,-0.976904,-0.131678,0.182595,-0.984224,-0.017629,-0.257675,6


In [ ]:
train_data.interpolate(linear='time', inplace=True) # 결측치를 (이전 값 +  이후 값) / 2로 반영

In [ ]:
train_data.head()

,A_x,A_y,A_z,B_x,B_y,B_z,label
timestamp,,,,,,,
2000-01-01 00:00:00.000,-0.988758,-0.144844,0.183325,-0.991851,-0.022659,-0.306466,6
2000-01-01 00:00:00.020,-1.008465,-0.143947,0.156743,-0.976224,-0.040058,-0.277258,6
2000-01-01 00:00:00.040,-0.991936,-0.143049,0.156618,-0.949758,-0.035796,-0.299548,6
2000-01-01 00:00:00.060,-1.007975,-0.140831,0.155302,-1.006806,-0.013057,-0.289020,6
2000-01-01 00:00:00.080,-0.991991,-0.159979,0.166965,-0.986998,0.015054,-0.293883,6


In [ ]:
# 다시 결측치를 확인해본다. 결측치가 대체되었는지 재확인한다.
train_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100000 entries, 2000-01-01 00:00:00 to 2019-01-12 02:02:02.100000
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   A_x     100000 non-null  float64
 1   A_y     100000 non-null  float64
 2   A_z     100000 non-null  float64
 3   B_x     100000 non-null  float64
 4   B_y     100000 non-null  float64
 5   B_z     100000 non-null  float64
 6   label   100000 non-null  int64  
dtypes: float64(6), int64(1)
memory usage: 6.1 MB


* 시계열 데이터를 반영해 결측치를 제거했으니 이제 'timestamp' 변수는 삭제한다.

In [ ]:
# 인덱스 재설정
train_data.reset_index(drop=True, inplace=True)

* 그냥 한번 모델을 만들어서 변수 중요도를 확인한다. 6개 중에 상위 4개만 확인하련다.
  * 보통 평타 이상을 친다는 랜덤 포레스트(Random Forest)를 만들어서 성능 평가를 해본다.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# X(feature)와 y(target)으로 구분한다.
X = train_data.drop(['label'], axis=1)
y = train_data['label']

In [ ]:
X.head()

,A_x,A_y,A_z,B_x,B_y,B_z
0,-0.988758,-0.144844,0.183325,-0.991851,-0.022659,-0.306466
1,-1.008465,-0.143947,0.156743,-0.976224,-0.040058,-0.277258
2,-0.991936,-0.143049,0.156618,-0.949758,-0.035796,-0.299548
3,-1.007975,-0.140831,0.155302,-1.006806,-0.013057,-0.289020
4,-0.991991,-0.159979,0.166965,-0.986998,0.015054,-0.293883


In [ ]:
# train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=42)

In [ ]:
# print(train_X.shape, test_X.shape, train_y.shape, test_y.shape)

* 우리 표준화를 할 것 입니다.
  * MinMax 말고 StandardScaler 써봐요

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# ss = StandardScaler()
# train_X = pd.DataFrame(ss.fit_transform(train_X), columns=['A_x','A_y', 'A_z','B_x','B_y', 'B_z']) # 저는 2차원 넘파이 배열로 하는게 싫어서 데이터 프레임으로 만드는 겁니다.
# test_X = pd.DataFrame(ss.transform(test_X), columns=['A_x','A_y', 'A_z','B_x','B_y', 'B_z']) # 저는 2차원 넘파이 배열로 하는게 싫어서 데이터 프레임으로 만드는 겁니다.

In [ ]:
ss = StandardScaler()
X = pd.DataFrame(ss.fit_transform(X), columns = ['A_x',	'A_y',	'A_z',	'B_x',	'B_y', 'B_z'])

* 모델을 만들어봐요

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=200, max_depth=10)
rf.fit(X, y)

RandomForestClassifier(max_depth=10, n_estimators=200)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
# print('훈련 세트 점수 : ', rf.score(train_X, train_y))
# print('평가 세트 점수 : ', rf.score(test_X, test_y))

* 변수 중요도를 확인해본다.

In [ ]:
# 변수의 특성 중요도 계산하기
# def plot_feature_importance(importance, names, result_only = False, topn = 'all'):
#     feature_importance = np.array(importance)
#     feature_name = np.array(names)

#     data={'feature_name':feature_name,'feature_importance':feature_importance}
#     fi_temp = pd.DataFrame(data)

#     #변수의 특성 중요도 순으로 정렬하기
#     fi_temp.sort_values(by=['feature_importance'], ascending=False,inplace=True)
#     fi_temp.reset_index(drop=True, inplace = True)

#     if topn == 'all' :
#         fi_df = fi_temp.copy()
#     else :
#         fi_df = fi_temp.iloc[:topn]

#     #변수의 특성 중요도 그래프로 그리기
#     if result_only == False :
#         plt.figure(figsize=(10,20))
#         sns.barplot(x='feature_importance', y='feature_name', data = fi_df)

#         plt.xlabel('importance')
#         plt.ylabel('feature name')
#         plt.grid()

#     return fi_df

In [ ]:
# plot_feature_importance(rf.feature_importances_, train_X.columns, result_only=True)

* 네 저는 앞으로 모델링할 떄 'A_z', 'B_x', 'B_z', 'A_x'만 사용하겠습니다.

<h1> 본격적인 모델링을 시작합시다. </h1>

![](https://github.com/DA4BAM/image/blob/main/pipeline%20function.png?raw=true)

* 우리 이런 흐름으로 갈 것입니다. 물론 label이 11개여서 아주 세세하게 따지면 다른 부분이 존재하겠죠?

* 하지만 우리는 이런 흐름으로 갈 거예요

In [ ]:
train_data.head()

,A_x,A_y,A_z,B_x,B_y,B_z,label
0,-0.988758,-0.144844,0.183325,-0.991851,-0.022659,-0.306466,6
1,-1.008465,-0.143947,0.156743,-0.976224,-0.040058,-0.277258,6
2,-0.991936,-0.143049,0.156618,-0.949758,-0.035796,-0.299548,6
3,-1.007975,-0.140831,0.155302,-1.006806,-0.013057,-0.289020,6
4,-0.991991,-0.159979,0.166965,-0.986998,0.015054,-0.293883,6


* 우리 'A_z', 'B_x', 'B_z', 'A_x'만 사용한다.

In [ ]:
# train_data = train_data[['A_z', 'B_x', 'B_z', 'A_x', 'label']]

In [ ]:
# train_data.head()

* 정적행동(0), 동적행동(1)으로 나눈다.

* label을 바탕으로 저는 이렇게 동적 행동(1)과 정적 행동(1)으로 구분하였습니다.
  * 1: 걷기 -> 동적 행동(1)로 정의
  * 2. 뛰기 -> 동적 행동(1)로 정의
  * 3. 천천히 걷기 -> 동적 행동(1)로 정의
  * 4. 계단 오르기 -> 동적 행동(1)로 정의
  * 5. 계단 내려가기 -> 동적 행동(1)로 정의
  * 6. 서있기 -> 정적 행동(0)으로 정의
  * 7. 앉아있기 -> 정적 행동(0)으로 정의
  * 8. 누워있기 -> 정적 행동(0)으로 정의
  * 9. 자전거 타기 -> 동적 행동(1)로 정의
  * 10. 서서 자전거 타기 -> 동적 행동(1)로 정의
  * 11. 자전거에 앉아 있기 -> 정적 행동(0)으로 정의

In [ ]:
# is_dynamic 변수 추가
# train_data['is_dynamic'] = train_data['label'].apply(lambda label: 0 if label in [6, 7, 8, 11] else 1)

* 우리 train_data에서 X, y1(label), y2(is_dynamic)를 구분해요

In [ ]:
# train_X = train_data[['A_z', 'B_x', 'B_z', 'A_x', 'A_y', 'B_y']]
# train_y1 = train_data['label']
# train_y2 = train_data['is_dynamic']

* 표준화를 진행한다.
  * 일단 학습 데이터만 보일 것인데
  * 나중에 추가로 테스트 데이터도 표준화할 예정입니다.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# sds = StandardScaler()
# train_X = pd.DataFrame(sds.fit_transform(train_X), columns = ['A_z', 'B_x', 'B_z', 'A_x', 'A_y', 'B_y' ]) # 저는 2차원 넘파이 배열로 만들기 싫어서 데이터프레임으로 만들겠습니다.

* 정적 행동(0)과 동적 행동(1)를 구분하는 모델1를 만들어요!

In [ ]:
# model1 = RandomForestClassifier(n_estimators=300, max_depth=5, n_jobs=-1)
# model1.fit(train_X, train_y2)

* 정적 행동(0)으로 판단했으면 정적 행동 중에서 어떤 정적 행동을 구분하는지에 대한 모델 2-1를 만들어요!

In [ ]:
# train_static_data = train_data.loc[train_data['label'].isin([6, 7, 8, 11]), :]

In [ ]:
# train_static_data.head()

In [ ]:
# train_static_X = train_static_data.drop(['label', 'is_dynamic'], axis=1)
# train_static_y = train_static_data['label']

In [ ]:
# model2_1 = RandomForestClassifier(n_estimators=300, max_depth=5, n_jobs=-1)
# model2_1.fit(train_static_X, train_static_y)

In [ ]:
# train_static_X.columns

* 동적 행동(1)로 판단했으면 동적 행동 중에서 어떤 동적 행동을 구분하는지에 대한 모델 2-2를 만들어요!

In [ ]:
# train_dynamic_data = train_data.loc[train_data['label'].isin([1, 2, 3, 4, 5, 9, 10]), :]

In [ ]:
# train_dynamic_X = train_dynamic_data.drop(['label', 'is_dynamic'], axis=1)
# train_dynamic_y = train_dynamic_data['label']

In [ ]:
# model2_2 = RandomForestClassifier(n_estimators=300, max_depth=5, n_jobs=-1)
# model2_2.fit(train_dynamic_X, train_dynamic_y)

* 모델 1, 모델 2-1, 모델 2-2를 다 만들었다.

* 이제 테스트 데이터를 가져와서 예측을 해볼까요?

<h1> 테스트 데이터를 이용하여 예측을 해봐요 </h1>

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/미니프로젝트5차part2_Kaggle/test.csv')

In [ ]:
test_data.shape

(13234, 8)

In [ ]:
test_data.columns

Index(['Unnamed: 0', 'timestamp', 'A_x', 'A_y', 'A_z', 'B_x', 'B_y', 'B_z'], dtype='object')

In [ ]:
# test 데이터에 'Unamed : 0', 'timestamp' 변수를 삭제한다.
test_data.drop(['Unnamed: 0', 'timestamp'], axis=1, inplace=True)

In [ ]:
test_data.head()

,A_x,A_y,A_z,B_x,B_y,B_z
0,-1.000957,-0.170691,0.124889,-0.979561,0.003150,-0.264673
1,-0.874830,0.132696,-0.501727,-1.274911,0.045122,0.121270
2,-1.219112,0.074678,0.435331,-0.860820,0.222740,0.008689
3,-0.907752,-0.171816,0.211507,-0.972017,0.337799,1.013534
4,-1.031261,0.000340,-0.091693,-0.217434,-0.323466,0.931614


* 우리는 변수를 'A_z', 'B_x', 'B_z', 'A_x'만 사용할 것 이니 테스트 데이터에도 똑같이 적용합시다.

In [ ]:
test_data.head()

,A_x,A_y,A_z,B_x,B_y,B_z
0,-1.000957,-0.170691,0.124889,-0.979561,0.003150,-0.264673
1,-0.874830,0.132696,-0.501727,-1.274911,0.045122,0.121270
2,-1.219112,0.074678,0.435331,-0.860820,0.222740,0.008689
3,-0.907752,-0.171816,0.211507,-0.972017,0.337799,1.013534
4,-1.031261,0.000340,-0.091693,-0.217434,-0.323466,0.931614


In [ ]:
test_data = pd.DataFrame(ss.transform(test_data), columns = ['A_x',	'A_y',	'A_z',	'B_x',	'B_y',	'B_z']) # 저는 2차원 넘파이 배열로 만들기 싫어서 데이터프레임으로 만들겠습니다.

In [ ]:
test_data.columns

Index(['A_x', 'A_y', 'A_z', 'B_x', 'B_y', 'B_z'], dtype='object')

In [ ]:
predict = list(rf.predict(test_data))

In [ ]:
submit =  pd.DataFrame({
    'ID': list(range(0, 13234, 1)),
    'label': predict
})

In [ ]:
submit['label'].value_counts()

1     1768
8     1533
2     1495
10    1422
7     1406
11    1322
9     1166
4     1147
3     1016
6      632
5      327
Name: label, dtype: int64

* 모델 1를 넣어봅시다.

In [ ]:
# model1_predict = list(model1.predict(test_data))

# print('0으로 예측한 개수 : ', model1_predict.count(0))
# print('1로 예측한 개수 : ', model1_predict.count(1))

In [ ]:
# test_data.head()

* model1_predict를 가지고 0이냐 1이냐에 따라서 모델 2-1, 모델 2-2를 통해 최종 예측을 시도한다.

In [ ]:
# 모델 2-1를 통해 최종 예측 하는 함수
# def static_predict(idx):
#     result = model2_1.predict(test_data.iloc[idx].values.reshape(1, -1)) # Series를 2차원 넘파이 배열로 왜냐하면 머신러닝, 딥러닝 입력 형태가 데이터프레임이나 2차원 넘파이 배열이기 떄문
#     return result[0]

In [ ]:
# 모델 2-2를 통해 최종 예측 하는 함수
# def dynamic_predict(idx):
#     result = model2_2.predict(test_data.iloc[idx].values.reshape(1, -1)) # Series를 2차원 넘파이 배열로 왜냐하면 머신러닝, 딥러닝 입력 형태가 데이터프레임이나 2차원 넘파이 배열이기 떄문
#     return result[0]

In [ ]:
# 최종 예측된 것을 담는 리스트
# model2_predict = []

In [ ]:
# model1_predict 된 것을 하나씩 for문으로 돌린다.
# for idx, m1p in enumerate(model1_predict):
#     if m1p == 0:
#         result = static_predict(idx)
#         model2_predict.append(result)
#     else:
#         result = dynamic_predict(idx)
        # model2_predict.append(result)

<h1> Kaggle 제출 형태로 만든다 </h1>

In [ ]:
# Kaggle 제출 형태가 어떤지 확인한다.
sample = pd.read_csv('/content/drive/MyDrive/미니프로젝트5차part2_Kaggle/sample.csv')

In [ ]:
# submit =  pd.DataFrame({
#     'ID': list(range(0, 13234, 1)),
#     'label': model2_predict,
# })

In [ ]:
# Kaggle에 제출할 csv 파일 저장
# submit.to_csv('/content/drive/MyDrive/미니프로젝트5차part2_Kaggle/predict.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/미니프로젝트5차part2_Kaggle/영렬님 help/prediction_results (16)(영렬님 최종 예측 label 빈도수).csv')

In [ ]:
df['label'].value_counts(ascending=True)

5      693
6      703
4      946
3     1044
9     1192
11    1265
10    1355
7     1406
1     1474
8     1539
2     1617
Name: label, dtype: int64

In [ ]:
submit['label'].value_counts(ascending=True)

5      327
6      632
3     1016
4     1147
9     1166
11    1322
7     1406
10    1422
2     1495
8     1533
1     1768
Name: label, dtype: int64